# Pruebas Object Detection

In [18]:
import cv2
import numpy as np
import math
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors


In [19]:
# img = cv2.imread('bus.jpg')
model = YOLO('yolov8n.pt')
names = model.names
person_clss  = 0
print(names[person_clss])

person


## VisionEye View With Distance Calculation 
Source: https://docs.ultralytics.com/guides/vision-eye/#samples

In [41]:
cap = cv2.VideoCapture("personas.mp4")

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

print(f"Video frame width: {w}, height: {h}, fps: {fps}")
# out = cv2.VideoWriter('visioneye-distance-calculation.avi', cv2.VideoWriter_fourcc(*'MJPG'), fps, (w, h))

center_point = (0, h)
pixel_per_meter = 10

txt_color, txt_background, bbox_clr = ((0, 0, 0), (255, 255, 255), (255, 0, 255))

while True:
    ret, im0 = cap.read()
    if not ret:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    annotator = Annotator(im0, line_width=2)

    results = model.track(im0, persist=True)
    results.plot()
    boxes = results[0].boxes.xyxy.cpu()


    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, track_id in zip(boxes, track_ids):
            annotator.box_label(box, label=str(track_id), color=bbox_clr)
            annotator.visioneye(box, center_point)

            x1, y1 = int((box[0] + box[2]) // 2), int((box[1] + box[3]) // 2)    # Bounding box centroid

            distance = (math.sqrt((x1 - center_point[0]) ** 2 + (y1 - center_point[1]) ** 2))/pixel_per_meter

            text_size, _ = cv2.getTextSize(f"Distance: {distance:.2f} m", cv2.FONT_HERSHEY_SIMPLEX,1.2, 3)
            cv2.rectangle(im0, (x1, y1 - text_size[1] - 10),(x1 + text_size[0] + 10, y1), txt_background, -1)
            cv2.putText(im0, f"Distance: {distance:.2f} m",(x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2,txt_color, 3)

    # out.write(im0)
    cv2.imshow("visioneye-distance-calculation", im0)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# out.release()
cap.release()
cv2.destroyAllWindows()


Video frame width: 0, height: 0, fps: 0
Video frame is empty or video processing has been successfully completed.


## Distance Calculation

Source: 
* https://github.com/Asadullah-Dal17/Yolov4-Detector-and-Distance-Estimator/blob/master/DistanceEstimation.py
* https://www.youtube.com/watch?v=FcRCwTgYXJw&ab_channel=AiPhile 

### Funciones Auxiliares

In [15]:
def focal_length_finder(measured_distance, real_width, width_in_frame):
    focal_length = (width_in_frame * measured_distance) / real_width
    return focal_length

In [14]:
def distance_finder(focal_length, real_width, width_in_frame):
    distance = (real_width * focal_length) / width_in_frame
    return distance

### Detección y Calculo de distancia en Video

In [80]:
# Medidas en metros
KNOWN_DISTANCE = 2
PERSON_WIDTH = 0.38 # Ancho de una persona promedio - Medidas de Fri - 38 cm
MODE = "calibration"
VIDEO_PATH = "//home/fcanof/Documents/sdv_vision_notebooks/data/ref_video/2_personas.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
count = 0
while True:
    ret, frame = cap.read()

    results = model(frame, classes=0)
    if results[0].boxes is not None:
        boxes_w = results[0].boxes.xywh.cpu()
        boxes_xyxy = results[0].boxes.xyxy.cpu()
        for box_w, box_xyxy in zip(boxes_w, boxes_xyxy):
            print("Mode: ", MODE)
            person_width = box_w[2]
            person_height = box_w[3]
            x,y = int(box_xyxy[0]), int(box_xyxy[1]+(person_height-50))
            if MODE == "calibration":
                focal_person = focal_length_finder(KNOWN_DISTANCE, PERSON_WIDTH, person_width)
                print(f"Focal length: {focal_person}")
            else:
                distance = distance_finder(focal_person, PERSON_WIDTH, person_width)
                distance = round(float(distance), 3)
                print(f"Distance: {distance} meters")
            cv2.rectangle(frame, (x, y-1), (x+200, y+25), (0,0,255),-1 )
            text = "Distance "+str(distance)+" meters"
            cv2.putText(frame, text, (x+10,y+20), cv2.FONT_HERSHEY_SIMPLEX, 0.48, (255,255,255), 2)

    cv2.imshow('frame',frame)
    count += 1
    if count == 100:
        MODE = "distance"
    key = cv2.waitKey(1)
    if key ==ord('q'):
        break
cv2.destroyAllWindows()
cap.release()




0: 384x640 1 person, 80.4ms
Speed: 4.5ms preprocess, 80.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Mode:  calibration
Focal length: 2695.093994140625

0: 384x640 1 person, 140.3ms
Speed: 4.9ms preprocess, 140.3ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)
Mode:  calibration
Focal length: 2966.550537109375

0: 384x640 1 person, 78.9ms
Speed: 2.7ms preprocess, 78.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
Mode:  calibration
Focal length: 3079.9150390625

0: 384x640 1 person, 71.0ms
Speed: 2.4ms preprocess, 71.0ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Mode:  calibration
Focal length: 2589.5810546875

0: 384x640 2 persons, 72.5ms
Speed: 2.6ms preprocess, 72.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
Mode:  calibration
Focal length: 3044.8251953125
Mode:  calibration
Focal length: 3853.72265625

0: 384x640 2 persons, 73.3ms
Speed: 1.8ms preprocess, 73.3ms inference,

error: OpenCV(4.6.0) /io/opencv/modules/highgui/src/window.cpp:967: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'
